# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 15

In [2]:
import sys

In [3]:
sys.path.append('..')

In [4]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson
from py_files.fbref_data import fetch_player_data, fetch_team_data
from pathlib import Path

# allow more data columns to be shown than by default
pd.set_option('max_columns',500)

In [5]:
# download player data from fbref.com
playerStats = fetch_player_data()
# save player stats
path = Path(f'../data/fbref/player_stats_week{latest_gameweek}.csv')
playerStats.to_csv(path)
playerStats.head()

General                                                               \
       Rk           Player   Nation Pos            Squad     Age  Born   
0       1       Max Aarons  eng ENG  DF     Norwich City  21-339  2000   
1       2        Che Adams  sct SCO  FW      Southampton  25-149  1996   
2       3  Rayan Aït Nouri   fr FRA  DF           Wolves  20-186  2001   
3       4  Kristoffer Ajer   no NOR  DF        Brentford  23-236  1998   
4       5       Nathan Aké   nl NED  DF  Manchester City  26-294  1995   

  Playing Time                    Performance                              \
            MP Starts   Min   90s         Gls Ast G-PK PK PKatt CrdY CrdR   
0           15     15  1350  15.0           0   1    0  0     0    3    0   
1           12     11   858   9.5           3   1    3  0     0    0    0   
2            8      7   628   7.0           0   1    0  0     0    1    0   
3            7      6   565   6.3           0   0    0  0     0    1    0   
4            5      4   375   4.2           0   0    0  0     0    0    0   

  Per 90 Minutes                          Expected                    \
             Gls   Ast   G+A  G-PK G+A-PK       xG npxG   xA npxG+xA   
0           0.00  0.07  0.07  0.00   0.07      0.6  0.6  0.9     1.4   
1           0.31  0.10  0.42  0.31   0.42      2.4  2.4  1.4     3.8   
2           0.00  0.14  0.14  0.00   0.14      0.3  0.3  0.5     0.8   
3           0.00  0.00  0.00  0.00   0.00      0.2  0.2  0.3     0.5   
4           0.00  0.00  0.00  0.00   0.00      0.2  0.2  0.0     0.2   

  Per 90 Minutes                            General  
              xG    xA xG+xA  npxG npxG+xA  Matches  
0           0.04  0.06  0.10  0.04    0.10  Matches  
1           0.25  0.15  0.40  0.25    0.40  Matches  
2           0.04  0.08  0.12  0.04    0.12  Matches  
3           0.03  0.05  0.07  0.03    0.07  Matches  
4           0.05  0.00  0.05  0.05    0.05  Matches

In [6]:
# download team data from fbref.com
teamStats = fetch_team_data()
teamStats.set_index('Squad', inplace=True)
# save team stats
#path = Path(f'../data/fbref/team_stats_week{latest_gameweek}.csv')
#teamStats.to_csv(path)
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Manchester City,1,15,11,2,2,32,9,23,35,33.2,10.1,23.1,1.54,W W W W W,52565,Bernardo Silva - 7,Ederson,NaN
Liverpool,2,15,10,4,1,44,12,32,34,39.0,14.4,24.6,1.64,L W W W W,52918,Mohamed Salah - 13,Alisson,NaN
Chelsea,3,15,10,3,2,35,9,26,33,27.1,15.3,11.8,0.79,D W D W L,39862,Mason Mount - 5,Edouard Mendy,NaN
West Ham,4,15,8,3,4,28,19,9,27,21.4,17.1,4.3,0.29,W L L D W,58624,Michail Antonio - 6,Łukasz Fabiański,NaN
Tottenham,5,14,8,1,5,16,17,-1,25,17.1,16.6,0.5,0.04,L D W W W,57463,Son Heung-min - 6,Hugo Lloris,NaN
Manchester Utd,6,15,7,3,5,25,24,1,24,20.8,24.8,-4.0,-0.27,L L D W W,72998,Cristiano Ronaldo - 6,David de Gea,NaN
Arsenal,7,15,7,2,6,18,22,-4,23,19.0,23.9,-5.0,-0.33,W L W L L,59334,Emile Smith-Rowe - 5,Aaron Ramsdale,NaN
Wolves,8,15,6,3,6,12,13,-1,21,16.1,18.6,-2.4,-0.16,L W D D L,30477,Hwang Hee-chan - 4,José Sá,NaN
Brighton,9,15,4,8,3,14,16,-2,20,17.2,18.6,-1.4,-0.09,D L D D D,30861,Neal Maupay - 6,Robert Sánchez,NaN


In [7]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,75.0,NaN,80201,0,0,-4,4,0,1,0.4,0.5,0,Bernd,0.0,False,Groin Injury - 75% chance of playing,2021-12-06T23:00:13.342729Z,46,80201.jpg,1.3,Leno,0.9,False,None,d,1,3,4,60463,273,176598,1485,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,296,25,588,62,572,56,366,25,NaN,,NaN,,NaN,
2,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,0.7,False,None,u,1,3,0,19017,0,66527,554,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,462,42,450,23,412,14,467,42,NaN,,NaN,,NaN,
3,0.0,0.0,47431,0,0,-2,2,0,3,0.0,0.0,0,Willian,0.0,False,Transferred to Corinthians,2021-08-20T09:30:14.065783Z,63,47431.jpg,0.0,Borges Da Silva,0.1,False,None,u,1,3,0,914,0,20262,21,0.0,0.0,Willian,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,636,260,635,260,635,260,636,260,NaN,,NaN,,NaN,
4,100.0,100.0,54694,0,0,-3,3,0,4,2.0,2.0,1,Pierre-Emerick,1.5,False,,2021-11-19T16:30:08.907928Z,97,54694.jpg,3.1,Aubameyang,2.8,False,None,a,1,3,44,717455,8480,740075,23640,0.2,4.5,Aubameyang,1036,4,1,6,16,0,0,2,3,0,0,7,131,217.6,132.4,582.0,92.9,115,14,128,21,6,3,27,10,NaN,,NaN,,1.0,
5,NaN,NaN,58822,0,0,-3,3,0,2,0.5,0.5,0,Cédric,0.0,False,,None,42,58822.jpg,0.3,Soares,0.2,False,None,a,1,3,1,11782,183,22976,228,0.0,0.2,Cédric,188,0,0,0,7,0,0,0,1,0,0,0,21,30.8,36.1,1.0,6.8,362,137,279,87,389,145,374,134,NaN,,NaN,,NaN,


In [8]:
# list all team names
team_names = ['Arsenal', 'Aston Villa', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton',
        'Leicester City', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 
        'Norwich City', 'Southampton', 'Tottenham', 'Watford', 'West Ham', 'Wolves']
# assign proper team names for each player
df['team_name'] = [team_names[i] for i in df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [9]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester City,1,15,11,2,2,32,9,23,35,33.2,10.1,23.1,1.54,W W W W W,52565,Bernardo Silva - 7,Ederson,NaN,2.213333,0.673333,0.510006
Liverpool,2,15,10,4,1,44,12,32,34,39.0,14.4,24.6,1.64,L W W W W,52918,Mohamed Salah - 13,Alisson,NaN,2.600000,0.960000,0.382893
Chelsea,3,15,10,3,2,35,9,26,33,27.1,15.3,11.8,0.79,D W D W L,39862,Mason Mount - 5,Edouard Mendy,NaN,1.806667,1.020000,0.360595
West Ham,4,15,8,3,4,28,19,9,27,21.4,17.1,4.3,0.29,W L L D W,58624,Michail Antonio - 6,Łukasz Fabiański,NaN,1.426667,1.140000,0.319819
Tottenham,5,14,8,1,5,16,17,-1,25,17.1,16.6,0.5,0.04,L D W W W,57463,Son Heung-min - 6,Hugo Lloris,NaN,1.221429,1.185714,0.305528
Manchester Utd,6,15,7,3,5,25,24,1,24,20.8,24.8,-4.0,-0.27,L L D W W,72998,Cristiano Ronaldo - 6,David de Gea,NaN,1.386667,1.653333,0.191411
Arsenal,7,15,7,2,6,18,22,-4,23,19.0,23.9,-5.0,-0.33,W L W L L,59334,Emile Smith-Rowe - 5,Aaron Ramsdale,NaN,1.266667,1.593333,0.203247
Wolves,8,15,6,3,6,12,13,-1,21,16.1,18.6,-2.4,-0.16,L W D D L,30477,Hwang Hee-chan - 4,José Sá,NaN,1.073333,1.240000,0.289384
Brighton,9,15,4,8,3,14,16,-2,20,17.2,18.6,-1.4,-0.09,D L D D D,30861,Neal Maupay - 6,Robert Sánchez,NaN,1.146667,1.240000,0.289384


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [10]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
#df['games played'] = df['total_points']/df['points_per_game']
#xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
# from last season
xA_proportion = 0.743

In [11]:
if latest_gameweek>1:
    df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
elif latest_gameweek == 1:
    df_previous_week = pd.read_csv('../data/fpl/pre_season.csv', index_col=0)
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week2,assists_week3,assists_week4,assists_week5,assists_week6,assists_week7,assists_week8,assists_week9,bonus,bonus_week1,bonus_week10,bonus_week11,bonus_week12,bonus_week13,bonus_week14,bonus_week2,bonus_week3,bonus_week4,bonus_week5,bonus_week6,bonus_week7,bonus_week8,bonus_week9,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week10,cleansheet_week11,cleansheet_week12,cleansheet_week13,cleansheet_week14,cleansheet_week2,cleansheet_week3,cleansheet_week4,cleansheet_week5,cleansheet_week6,cleansheet_week7,cleansheet_week8,cleansheet_week9,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,form 10,form 15,form 20,form 5,games played,gameweek,gameweek 10 prediction,gameweek 11 prediction,gameweek 12 prediction,gameweek 13 prediction,gameweek 14 prediction,gameweek 15 prediction,gameweek 16 prediction,gameweek 17 prediction,gameweek 18 prediction,gameweek 19 prediction,gameweek 2 prediction,gameweek 20 prediction,gameweek 21 prediction,gameweek 22 prediction,gameweek 23 prediction,gameweek 3 prediction,gameweek 4 prediction,gameweek 5 prediction,gameweek 6 prediction,gameweek 7 prediction,gameweek 8 prediction,gameweek 9 prediction,goals_conceded,goals_scored,goals_week1,goals_week10,goals_week11,goals_week12,goals_week13,goals_week14,goals_week2,goals_week3,goals_week4,goals_week5,goals_week6,goals_week7,goals_week8,goals_week9,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,team_name,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value,value form 10,value form 5,valuePoints metric,value_form,value_next10,value_season,web_name,xA,xA_points,xA_week1,xA_week10,xA_week11,xA_week12,xA_week13,xA_week14,xA_week2,xA_week3,xA_week4,xA_week5,xA_week6,xA_week7,xA_week8,xA_week9,xG,xG+xA,xG+xA per game,xGA_week1,xGA_week10,xGA_week11,xGA_week12,xGA_week13,xGA_week14,xGA_week2,xGA_week3,xGA_week4,xGA_week5,xGA_week6,xGA_week7,xGA_week8,xGA_week9,xG_points,xG_week1,xG_week10,xG_week11,xG_week12,xG_week13,xG_week14,xG_week2,xG_week3,xG_week4,xG_week5,xG_week6,xG_week7,xG_week8,xG_week9,xPoints,xPoints week 1,xPoints week 10,xPoints week 11,xPoints week 12,xPoints week 13,xPoints week 14,xPoints week 2,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 24 prediction
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6.337758,2.112586,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,48,NaN,NaN,2.337758,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80201,NaN,NaN,0,0,-4,4,0.0,581,59,NaN,NaN,0,1,0.5,0.0,0,Bernd,0.0,NaN,1.799901,1.799901,NaN,3.0,14,NaN,NaN,NaN,NaN,NaN,3.912106,3.653806,3.653806,4.017078,4.221966,NaN,3.965480,3.636560,3.965480,4.059103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,7.9,355,24,False,79.0,286,24,270,NaN,NaN,46,0,0,NaN,0,NaN,80201.jpg,1.3,0,9,Leno,0.9,False,NaN,a,1,3,Arsenal,0.0,566,54,4,60019,269,174325,1022,0.4

In [12]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Chelsea,1,14,10,3,1,33,6,27,33,25.9,13.6,12.4,0.88,W D W D W,39862,"Reece James, Mason Mount - 4",Edouard Mendy,NaN,1.850000,0.971429,0.378542
Manchester City,2,14,10,2,2,29,8,21,32,29.5,9.4,20.1,1.44,L W W W W,52565,Bernardo Silva - 5,Ederson,NaN,2.107143,0.671429,0.510978
Liverpool,3,14,9,4,1,43,12,31,31,36.5,14.2,22.3,1.59,D L W W W,52918,Mohamed Salah - 13,Alisson,NaN,2.607143,1.014286,0.362661
West Ham,4,14,7,3,4,25,17,8,24,19.6,16.0,3.6,0.26,W W L L D,58436,Michail Antonio - 6,Łukasz Fabiański,NaN,1.400000,1.142857,0.318907
Arsenal,5,14,7,2,5,17,20,-3,23,17.9,22.9,-5.0,-0.36,W W L W L,59334,Emile Smith-Rowe - 5,Aaron Ramsdale,NaN,1.278571,1.635714,0.194813


In [13]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [14]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
bonus_column = 'bonus_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [15]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, ('Expected','xG')].values[0]  
    penalty_attempts = playerStats.loc[indicator,('Performance','PKatt')].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
                df.loc[ix, bonus_column] = df.loc[ix, 'bonus'] - df_previous_week.loc[ix, 'bonus']
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
                df.loc[ix, bonus_column] = df.loc[ix, 'bonus']
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [16]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [17]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, ('Expected','xA')].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [18]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [19]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        df.loc[ix,'games played'] = playerStats.loc[indicator,('Playing Time','MP')].values[0]
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            df.loc[ix,'games played'] = playerStats.loc[first_name_indicator,('Playing Time','MP')].values[0]
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            if name == 'gudmundsson':
                first_name = 'hann berg '
                exceptional_case_indicator = playerStats[('General','Player')].str.lower().str.contains(first_name)
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'gundogan':
                exceptional_case_indicator = \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('gundo')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'fabianski':
                exceptional_case_indicator = \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('fabia')) & \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('ski'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'soucek':
                exceptional_case_indicator = \
                    (playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('tom')) & \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('sou')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'fabiosilva':
                exceptional_case_indicator = \
                    playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                    .str.replace('ü', 'u').str.replace('ö', 'o').str.replace('ä', 'a').str.contains('biosilva')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'milivojevic':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('milivoje'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'emersonroyal':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('emerson')) & \
                                            (playerStats[('General','Squad')] == 'Tottenham')
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 's.armstrong':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('stuartarmstrong'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'placheta':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('przemys'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            elif name == 'dubravka':
                exceptional_case_indicator = (playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('bravka'))
                if playerStats.loc[exceptional_case_indicator].shape[0]==1:
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': no player found (exceptional_case_indicator fail).')
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            df.loc[ix,'games played'] = playerStats.loc[full_name_indicator,('Playing Time','MP')].values[0]
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats[('General','Squad')]==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                df.loc[ix,'games played'] = playerStats.loc[indicator & team_indicator,('Playing Time','MP')].values[0]
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats[('General','Player')].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    df.loc[ix,'games played'] = playerStats.loc[exceptional_case_indicator,('Playing Time','MP')].values[0]
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [20]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('bonus_week' + str(i)) 
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,team_name,games played,adjusted points,xG,xG_points,clean_sheet_points,xGA_week15,cleansheet_week15,xA,xA_points,xG_week15,goals_week15,xPoints week 15,bonus_week15,xA_week15,assists_week15,adjusted points per game,xPoints week 1,xG_week1,goals_week1,bonus_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,bonus_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,bonus_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,bonus_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,bonus_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,xPoints week 6,xG_week6,goals_week6,bonus_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6,xPoints week 7,xG_week7,goals_week7,bonus_week7,xGA_week7,cleansheet_week7,xA_week7,assists_week7,xPoints week 8,xG_week8,goals_week8,bonus_week8,xGA_week8,cleansheet_week8,xA_week8,assists_week8,xPoints week 9,xG_week9,goals_week9,bonus_week9,xGA_week9,cleansheet_week9,xA_week9,assists_week9,xPoints week 10,xG_week10,goals_week10,bonus_week10,xGA_week10,cleansheet_week10,xA_week10,assists_week10,xPoints week 11,xG_week11,goals_week11,bonus_week11,xGA_week11,cleansheet_week11,xA_week11,assists_week11,xPoints week 12,xG_week12,goals_week12,bonus_week12,xGA_week12,cleansheet_week12,xA_week12,assists_week12,xPoints week 13,xG_week13,goals_week13,bonus_week13,xGA_week13,cleansheet_week13,xA_week13,assists_week13,xPoints week 14,xG_week14,goals_week14,bonus_week14,xGA_week14,cleansheet_week14,xA_week14,assists_week14
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,75.0,NaN,80201,0,0,-4,4,0,1,0.4,0.5,0,Bernd,0.0,False,Groin Injury - 75% chance of playing,2021-12-06T23:00:13.342729Z,46,80201.jpg,1.3,Leno,0.9,False,None,d,1,3,4,60463,273,176598,1485,0.0,0.9,Leno,270,0,0,0,9,0,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,296,25,588,62,572,56,366,25,NaN,,NaN,,NaN,,Arsenal,3.0,6.438964,0.0,0.00,2.438964,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.146321,2.090127,0.0,0.0,0.0,1.3,0.0,0.0,0.0,2.220093,0.0,0.0,0.0,2.9,0.0,0.0,0.0,1.089483,0.0,0.0,0.0,3.8,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,4.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.8,0.0,NaN,NaN
2,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31T22:00:09.069158Z,40,115918.jpg,0.0,Rúnarsson,0.7,False,None,u,1,3,0,19017,0,66527,554,0.0,0.0,Rúnarsson,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,462,42,450,23,412,14,467,42,NaN,,NaN,,NaN,,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [21]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False).head(30)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
177,Calvert-Lewin,3.0,23,7.7,25.540000,8.513333
233,Salah,15.0,152,10.1,123.573393,8.238226
237,Alexander-Arnold,13.0,99,7.6,90.807430,6.985187
268,Torres,4.0,26,6.5,25.311023,6.327756
265,Foden,9.0,48,5.3,54.703051,6.078117
240,Jota,14.0,79,5.6,84.931500,6.066536
135,Chilwell,6.0,54,9.0,36.325279,6.054213
250,Gündogan,10.0,54,5.4,60.042057,6.004206
253,Mendy,1.0,3,3.0,5.940023,5.940023


In [22]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [23]:
total_xG = playerStats[('Expected','xG')].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 400
Total xG: 412.5
goals per xG: 0.9696969696969697
